Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using DynamicHMC, TransformVariables, LogDensityProblems, MCMCDiagnostics
using Parameters, ForwardDiff

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "05")
cd(ProjDir)

Read the milk data

In [3]:
wd = CSV.read(rel_path("..", "data", "milk.csv"), delim=';')
df = convert(DataFrame, wd);
dcc = filter(row -> !(row[:neocortex_perc] == "NA"), df)
dcc[:kcal_per_g] = convert(Vector{Float64}, dcc[:kcal_per_g])
dcc[:log_mass] = log.(convert(Vector{Float64}, dcc[:mass]))

17-element Array{Float64,1}:
  0.6678293725756554
  1.6582280766035324
  1.6808279085207734
  0.9202827531436925
 -0.3856624808119846
 -2.120263536200091 
 -0.7550225842780328
 -1.1394342831883648
  0.4382549309311553
  1.1755733298042381
  2.509599262378372 
  1.6808279085207734
  3.5689691574413787
  4.374876130645041 
  3.70721041079866  
  3.4998353515591547
  4.006423680849631 

Show first 5 rows

In [4]:
first(dcc[[3, 7, 9]], 5)

,kcal_per_g,mass,log_mass
,Float64,Float64⍰,Float64
1,0.49,1.95,0.667829
2,0.47,5.25,1.65823
3,0.56,5.37,1.68083
4,0.89,2.51,0.920283
5,0.92,0.68,-0.385662


Define the model struct

In [5]:
struct m_5_6{TY <: AbstractVector, TX <: AbstractMatrix}
    "Observations."
    y::TY
    "Covariates"
    X::TX
end

Make the type callable with the parameters *as a single argument*.

In [6]:
function (problem::m_5_6)(θ)
    @unpack y, X, = problem   # extract the data
    @unpack β, σ = θ            # works on the named tuple too
    ll = 0.0
    ll += logpdf(Normal(0, 100), X[1]) # a = X[1]
    ll += logpdf(Normal(0, 1), X[2]) # b1 = X[2]
    ll += logpdf(TDist(1.0), σ)
    ll += loglikelihood(Normal(0, σ), y .- X*β)
    ll
end

Instantiate the model with data and inits.

In [7]:
N = size(dcc, 1)
X = hcat(ones(N), dcc[:log_mass]);
y = dcc[:kcal_per_g]
p = m_5_6(y, X);
p((β = [1.0, 2.0], σ = 1.0))

-242.8761844035513

Write a function to return properly dimensioned transformation.

In [8]:
problem_transformation(p::m_5_6) =
    as((β = as(Array, size(p.X, 2)), σ = asℝ₊))
# Wrap the problem with a transformation, then use Flux for the gradient.
P = TransformedLogDensity(problem_transformation(p), p)
∇P = ADgradient(:ForwardDiff, P);

Tune and sample.

In [9]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000);

MCMC, adapting ϵ (75 steps)
0.0013 s/step ...done
MCMC, adapting ϵ (25 steps)
0.00016 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0018 s/step ...done
MCMC, adapting ϵ (100 steps)
2.8e-5 s/step ...done
MCMC, adapting ϵ (200 steps)
3.9e-5 s/step ...done
MCMC, adapting ϵ (400 steps)
4.3e-5 s/step ...done
MCMC, adapting ϵ (50 steps)
4.0e-5 s/step ...done
MCMC (1000 steps)
3.9e-5 s/step ...done


We use the transformation to obtain the posterior from the chain.

In [10]:
posterior = TransformVariables.transform.(Ref(∇P.transformation), get_position.(chain));
posterior[1:5]

5-element Array{NamedTuple{(:β, :σ),Tuple{Array{Float64,1},Float64}},1}:
 (β = [0.652621, -0.0317121], σ = 0.20869672216964608)
 (β = [0.751457, -0.0339145], σ = 0.14016880912125376)
 (β = [0.669239, -0.0202523], σ = 0.20449583034556648)
 (β = [0.770653, -0.044111], σ = 0.1780130958112069)  
 (β = [0.722116, -0.0300986], σ = 0.19847704992060286)

Extract the parameter posterior means: `β`,

In [11]:
posterior_β = mean(first, posterior)

2-element Array{Float64,1}:
  0.7066340426232676 
 -0.03141686102215767

then `σ`:

In [12]:
posterior_σ = mean(last, posterior)

0.1828900392231636

Effective sample sizes (of untransformed draws)

In [13]:
ess = mapslices(effective_sample_size,
                get_position_matrix(chain); dims = 1)
# NUTS-specific statistics
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.9, min/25%/median/75%/max: 0.02 0.87 0.95 0.99 1.0
  termination: AdjacentTurn => 26% DoubledTurn => 74%
  depth: 1 => 6% 2 => 57% 3 => 32% 4 => 4% 5 => 1%


cmdstan result

In [14]:
cmdstan_result = "
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean         SD        Naive SE       MCSE      ESS
    a  0.70472876 0.057040655 0.00090189195 0.0011398893 1000
   bm -0.03150330 0.023642759 0.00037382484 0.0004712342 1000
sigma  0.18378372 0.039212805 0.00062000888 0.0011395979 1000

Quantiles:
          2.5%       25.0%       50.0%        75.0%       97.5%
    a  0.59112968  0.66848775  0.70444950  0.741410500 0.81915225
   bm -0.07729257 -0.04708425 -0.03104865 -0.015942925 0.01424901
sigma  0.12638780  0.15605950  0.17800600  0.204319250 0.27993590
";

Extract the parameter posterior means: `[β, σ]`,

In [15]:
[posterior_β, posterior_σ]

2-element Array{Any,1}:
  [0.706634, -0.0314169]
 0.1828900392231636     

End of `05/5.6d.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*